In [59]:
"""
Führt Regressionsanalyse durch

"""
import pandas as pd
from sklearn import linear_model

#Initmessage
print ("starting Regression") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )


print("data collected")

print(data_frame_investigated.head())

starting Regression
data collected
      slug symbol     name        date  ranknow    open    high     low  \
0  bitcoin    BTC  Bitcoin  2013-04-28        1  135.30  135.98  132.10   
1  bitcoin    BTC  Bitcoin  2013-04-29        1  134.44  147.49  134.00   
2  bitcoin    BTC  Bitcoin  2013-04-30        1  144.00  146.93  134.05   
3  bitcoin    BTC  Bitcoin  2013-05-01        1  139.00  139.89  107.72   
4  bitcoin    BTC  Bitcoin  2013-05-02        1  116.38  125.60   92.28   

    close  volume        market  close_ratio  spread  
0  134.21     0.0  1.500520e+09       0.5438    3.88  
1  144.54     0.0  1.491160e+09       0.7813   13.49  
2  139.00     0.0  1.597780e+09       0.3843   12.88  
3  116.99     0.0  1.542820e+09       0.2882   32.17  
4  105.21     0.0  1.292190e+09       0.3881   33.32  


In [60]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe

#set Date as Index


print(data_frame_investigated.head())


      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  
0  1.500520e+09       0.5438    3.88  
1  1.491160e+09       0.7813   13.49  
2  1.597780e+09       0.3843   12.88  
3  1.542820e+09       0.2882   32.17  
4  1.292190e+09       0.3881   33.32  


In [103]:
from statsmodels.formula.api import ols #library fuer regression

#dependency :patsy

def cryptoregress(crypto_id, feature_to_compare,crypto_id_feature_name="slug"):
    """
    lineare Regression zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    
   
    print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
    ).reset_index()
    
 
    
    data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    print(data_frame_id_exists)

    #Regressionsmodell machen
    regression_model = linear_model.LinearRegression(
        fit_intercept=True, normalize=True, copy_X=True, n_jobs=-1)


        #regressionsmodell fitten
    return regression_model.fit(
        data_frame_id_exists.drop(crypto_id,axis=1),
        data_frame_id_exists
    )      
    


regression_result=cryptoregress('bitcoin','close')

    
    
    

0    2013-04-28
1    2013-04-29
2    2013-04-30
3    2013-05-01
4    2013-05-02
Name: date, dtype: object
slug        0x  1337coin    1world     2give  300-token  42-coin   808coin  \
0     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
1     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
2     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
3     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
4     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
5     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
6     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
7     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
8     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
9     0.000000  0.000000  0.000000  0.000000       0.00      0.0  0.000000   
10    0.000000  0.000000  0.000000  

In [106]:
print(regression_result.intercept_)

[ 1.38777878e-16 -9.89334482e-19  1.76724954e-17 ... -9.99200722e-15
  1.38777878e-17 -3.90312782e-18]


In [109]:
max(regression_result.intercept_)

79.35872987435141